In [4]:
import numpy as np
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt
# import matplotlib as plt
import warnings

%matplotlib inline

In [5]:

train = pd.read_csv("data/train.csv")
test =  pd.read_csv("data/test.csv")


In [6]:
all_q = pd.concat([train[['question1', 'question2']], \
        test[['question1', 'question2']]], axis=0).reset_index(drop='index')

In [7]:
g = nx.Graph()
g.add_nodes_from(all_q.question1)
g.add_nodes_from(all_q.question2)



In [8]:
%%time
edges = list(all_q[['question1', 'question2']].to_records(index=False))
g.add_edges_from(edges)

CPU times: user 59.1 s, sys: 552 ms, total: 59.7 s
Wall time: 59.2 s


In [44]:
d = g.degree()
np.mean([d[k] for k in d])

0.062335352948153198

In [9]:
%%time
pr = nx.pagerank(g)

CPU times: user 5min 29s, sys: 6.74 s, total: 5min 36s
Wall time: 5min 36s


In [10]:
pr['Does Trump have a chance at winning presidency?']

1.7922898266984505e-07

In [13]:
import seaborn as sns

ImportError: No module named seaborn

In [16]:
# plt.hist(pr.values(),bins=100)
np.mean(pr.values()), np.min(pr.values()), np.max(pr.values())

(2.0881046524642131e-07, 3.1388169747735997e-08, 0.00045262659125822226)

In [17]:
def try_apply_dict(x,dict_to_apply):
    return dict_to_apply[x]

In [20]:
%%time
train['page_rank_q1'] = train.question1.apply(lambda x: try_apply_dict(x,pr))
train['page_rank_q2'] = train.question2.apply(lambda x: try_apply_dict(x,pr))

CPU times: user 624 ms, sys: 36 ms, total: 660 ms
Wall time: 600 ms


In [21]:
test['page_rank_q1'] = test.question1.apply(lambda x: try_apply_dict(x,pr))
test['page_rank_q2'] = test.question2.apply(lambda x: try_apply_dict(x,pr))

In [22]:
page_rank_train  = train[['page_rank_q1','page_rank_q2']]

In [23]:
page_rank_test  = test[['page_rank_q1','page_rank_q2']]

In [26]:
page_rank_train.to_csv("data/cache/page_rank_train.csv",index=False)
page_rank_test.to_csv("data/cache/page_rank_test.csv",index=False)
